In [56]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import IPython.display as ipd
import pandas as pd
import numpy as np
import seaborn as sns
import librosa
import soundfile as sf
import re
import scipy
import torch
import matplotlib.pyplot as plt
import nnAudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import gc
import seaborn as sns
from datasets import load_dataset

from glob import glob
from tqdm import tqdm
from nnAudio.features.mel import MelSpectrogram
from torchaudio.transforms import AmplitudeToDB
from transformers import (
    AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, 
    AutoProcessor, Wav2Vec2Model, HubertForCTC, 
    HubertModel, Data2VecAudioForCTC, Data2VecAudioModel,
    AutoModelForAudioXVector, Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
%matplotlib inline

DEVICE = "cuda"

# Dataset

In [33]:
ds = load_dataset('timit_asr', data_dir="../../data/TIMIT")
ds_train, ds_test = ds['train'], ds['test']

c:\Users\famil\AppData\Local\Programs\Python\Python38\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for timit_asr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/timit_asr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [9]:
print(ds_train[0])

{'file': 'c:\\DATA\\UCU\\YEAR_3\\AUDIO\\ucu_audio_processing_course\\data\\TIMIT\\data\\lisa\\data\\timit\\raw\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV', 'audio': {'path': 'c:\\DATA\\UCU\\YEAR_3\\AUDIO\\ucu_audio_processing_course\\data\\TIMIT\\data\\lisa\\data\\timit\\raw\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV', 'array': array([ 3.05175781e-05, -3.05175781e-05,  6.10351562e-05, ...,
       -3.05175781e-05, -1.52587891e-04, -2.44140625e-04]), 'sampling_rate': 16000}, 'text': 'She had your dark suit in greasy wash water all year.', 'phonetic_detail': {'start': [0, 3050, 4559, 5723, 6642, 8772, 9190, 10337, 11517, 12500, 12640, 14714, 15870, 16334, 18088, 20417, 21199, 22560, 22920, 23271, 24229, 25566, 27156, 28064, 29660, 31719, 33360, 33754, 34715, 36080, 36326, 37556, 39561, 40313, 42059, 43479, 44586], 'stop': [3050, 4559, 5723, 6642, 8772, 9190, 10337, 11517, 12500, 12640, 14714, 15870, 16334, 18088, 20417, 21199, 22560, 22920, 23271, 24229, 25566, 27156, 28064, 29660, 31719, 33360, 33754, 3

In [22]:
train_phonetics = [phone for x in ds_train for phone in x['phonetic_detail']['utterance']]
print("num of train phones:\t", len(set(train_phonetics)))

test_phonetics = [phone for x in ds_test for phone in x['phonetic_detail']['utterance']]
print("num of train phones:\t", len(set(test_phonetics)))

num of train phones:	 61


In [110]:
#source: https://www.kaggle.com/code/vitouphy/phoneme-recognition-with-wav2vec2
phon61_map = {
    'iy':'iy',  'ih':'ih',   'eh':'eh',  'ae':'ae',    'ix':'ih',  'ax':'ah',   'ah':'ah',  'uw':'uw',
    'ux':'uw',  'uh':'uh',   'ao':'aa',  'aa':'aa',    'ey':'ey',  'ay':'ay',   'oy':'oy',  'aw':'aw',
    'ow':'ow',  'l':'l',     'el':'l',  'r':'r',      'y':'y',    'w':'w',     'er':'er',  'axr':'er',
    'm':'m',    'em':'m',     'n':'n',    'nx':'n',     'en':'n',  'ng':'ng',   'eng':'ng', 'ch':'ch',
    'jh':'jh',  'dh':'dh',   'b':'b',    'd':'d',      'dx':'dx',  'g':'g',     'p':'p',    't':'t',
    'k':'k',    'z':'z',     'zh':'sh',  'v':'v',      'f':'f',    'th':'th',   's':'s',    'sh':'sh',
    'hh':'hh',  'hv':'hh',   'pcl':'h#', 'tcl':'h#', 'kcl':'h#', 'qcl':'h#','bcl':'h#','dcl':'h#',
    'gcl':'h#','h#':'h#',  '#h':'h#',  'pau':'h#', 'epi': 'h#','nx':'n',   'ax-h':'ah','q':'h#',
        "a": "ə", "ey": "eɪ", "aa": "ɑ", "ae": "æ", "ah": "ə", "ao": "ɔ",
        "aw": "aʊ", "ay": "aɪ", "ch": "ʧ", "dh": "ð", "eh": "ɛ", "er": "ər",
        "hh": "h", "ih": "ɪ", "jh": "ʤ", "ng": "ŋ",  "ow": "oʊ", "oy": "ɔɪ",
        "sh": "ʃ", "th": "θ", "uh": "ʊ", "uw": "u", "zh": "ʒ", "iy": "i", "y": "j"
}

def convert_phon61_to_phon39(sentence):
    tokens = [phon61_map[x] for x in sentence]
    return " ".join(tokens)

def normalize_phones(item):
    item['phonetic_detail']['utterance'] = convert_phon61_to_phon39(item['phonetic_detail']['utterance'])
    return item

In [111]:
ds_train_norm = ds_train.map(normalize_phones)
ds_test_norm = ds_test.map(normalize_phones)

In [113]:
train_phonetics_norm = [phone for x in ds_train_norm for phone in x['phonetic_detail']['utterance'].split()]
print("num of train phones:\t", len(set(train_phonetics_norm)))

test_phonetics_norm = [phone for x in ds_test_norm for phone in x['phonetic_detail']['utterance'].split()]
print("num of test phones:\t", len(set(test_phonetics_norm)))

num of train phones:	 47
num of test phones:	 47


In [122]:
vocab_list = sorted(list(set(train_phonetics_norm + test_phonetics_norm + ["UNK"])))
print(vocab_list)

['UNK', 'ah', 'aɪ', 'aʊ', 'b', 'd', 'dx', 'er', 'eɪ', 'f', 'g', 'h', 'h#', 'hh', 'i', 'ih', 'j', 'k', 'l', 'm', 'n', 'ng', 'oʊ', 'p', 'r', 's', 't', 'u', 'uw', 'v', 'w', 'z', 'æ', 'ð', 'ŋ', 'ɑ', 'ɔ', 'ɔɪ', 'ə', 'ər', 'ɛ', 'ɪ', 'ʃ', 'ʊ', 'ʒ', 'ʤ', 'ʧ', 'θ']


In [123]:
target_label_encoder = LabelEncoder().fit(vocab_list)

In [124]:
target_label_encoder.transform(ds_train_norm[0]["phonetic_detail"]["utterance"].split())

array([12, 42, 15, 13, 40, 12, 45, 41, 12,  5, 38, 12, 17, 25, 28, 12, 20,
       12, 10, 24, 15, 25, 15, 30, 36, 42, 12, 30, 36,  6,  7, 36, 18, 16,
       41,  7, 12])

In [125]:
train_y = [target_label_encoder.transform(ds_train_norm[i]["phonetic_detail"]["utterance"].split()) for i in range(len(ds_train_norm))]
test_y = [target_label_encoder.transform(ds_test_norm[i]["phonetic_detail"]["utterance"].split()) for i in range(len(ds_test_norm))]

# Evaluation metric (PER)

In [ ]:
def calculate_PER(reference, hypothesis):
    """
    Calculate Phoneme Error Rate (PER) between predicted and true phoneme sequences.    
    """
    n = len(reference)
    m = len(hypothesis)

    D = [[0] * (m + 1) for _ in range(n + 1)]
    
    for i in range(n + 1):
        D[i][0] = i
    for j in range(m + 1):
        D[0][j] = j
    
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if reference[i - 1] == hypothesis[j - 1]:
                D[i][j] = D[i - 1][j - 1]
            else:
                D[i][j] = min(D[i - 1][j] + 1,  # deletion
                              D[i][j - 1] + 1,  # insertion
                              D[i - 1][j - 1] + 1)  # substitution
    
    return D[n][m] / n

In [206]:
def compute_total_PER(predict_list, true_list):
    return np.mean([calculate_PER(predict_list[i], true_list[i]) for i in range(len(predict_list))])

In [147]:
reference_phonemes = [1, 2, 1]
hypothesis_phonemes = [1, 2, 3, 4]
PER = calculate_PER(reference_phonemes, hypothesis_phonemes)
print("Phoneme Error Rate:", PER)

Phoneme Error Rate: 0.6666666666666666


# Wav2Vec

In [12]:
# thanks god it finally works
from phonemizer.backend.espeak.wrapper import EspeakWrapper
_ESPEAK_LIBRARY = 'C:\Program Files\eSpeak NG\libespeak-ng.dll'
EspeakWrapper.set_library(_ESPEAK_LIBRARY)

In [156]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

Some weights of the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably

In [160]:
# tokenize
input_values = processor(ds_train[0]["audio"]["array"], return_tensors="pt", sampling_rate=16000).input_values

with torch.no_grad():
  logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

In [37]:
print(ds_train['text'][0])
print(ds_train_norm['phonetic_detail'][0]['utterance'])
print(transcription)

She had your dark suit in greasy wash water all year.
h# ʃ ih hh ɛ h# ʤ ɪ h# d ə h# k s uw h# n h# g r ih s ih w ɔ ʃ h# w ɔ dx er ɔ l j ɪ er h#
['ʃ iː h æ dʒ ɚ d ʌ k s uː t æ n ɡ ɹ iː s i w ɑː ʃ w ɔː ɾ ɚ ɹ ɔː l j ɪɹ']


In [130]:
transcription_unk = ["UNK" if x not in vocab_list else x for x in transcription[0].split()]
pred_y = target_label_encoder.transform(transcription_unk)

In [168]:
scores = calculate_PER(train_y[0], pred_y)
scores

0.7567567567567568

In [203]:
def predict_phonems(ds, sr=16000):
    phonemes = []
    for i in range(len(ds)):
        input_values = processor(ds[i]["audio"]["array"], return_tensors="pt", sampling_rate=sr).input_values
        with torch.no_grad():
            logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)
        transcription_unk = ["UNK" if x not in vocab_list else x for x in transcription[0].split()]
        pred_y = target_label_encoder.transform(transcription_unk)
        phonemes.append(pred_y)
    return phonemes

def pad_phonems(phonems):
    max_length = max(len(sublist) for sublist in phonems)
    X_padded = [torch.FloatTensor(list(sublist) + [0] * (max_length - len(sublist))) for sublist in phonems]
    return torch.cat(X_padded)

In [210]:
train_w2v = predict_phonems(ds_train_norm)
test_w2v = predict_phonems(ds_test_norm)

In [ ]:
classification_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression())
])

classification_pipeline.fit(train_w2v, train_y)

train_w2v2_predict = classification_pipeline.predict(train_w2v)
test_w2v2_predict = classification_pipeline.predict(test_w2v)

In [209]:
print("Train PER score: ", compute_total_PER(train_w2v, train_y))
print("Test PER score: ", compute_total_PER(test_w2v, test_y))

0.8023578413995871